In [28]:
import requests
from bs4 import BeautifulSoup
import unicodedata
import pandas as pd
import re
import time

In [29]:
data=pd.read_csv("../merged/len3_ndb_blp_DN_firm_filled.csv",encoding="utf-8",index_col=0)
data.head()


C:\Users\hakos\AppData\Local\Temp\ipykernel_17796\1964921763.py:1: DtypeWarning: Columns (118,119,131) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("../merged/len3_ndb_blp_DN_firm_filled.csv",encoding="utf-8",index_col=0)


,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価_x,後発品区分,総計,year,in_hospital,...,同一剤形・規格の後発医薬品がある先発医薬品,薬価_y,経過措置による使用期限,備考,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 4,_merge
0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,2.489539e+07,2014,1,...,○,13.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,1.024590e+07,2014,1,...,○,23.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
2,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,5.025912e+07,2014,0,...,○,13.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
3,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,2.742935e+07,2014,0,...,○,23.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,63.1,0.0,5.339043e+06,2014,1,...,○,63.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both


In [53]:
set(data["メーカー名"])

{' 陽進堂',
 ',テイコクメディックス',
 'Meiji Seika ファルマ',
 'Meファルマ',
 nan,
 '\u3000ダイト',
 '\u3000陽進堂',
 'あすか製薬',
 'あすかＡｃｔａｖｉｓ製薬',
 'あゆみ製薬',
 'そーせい',
 'わかもと製薬',
 'アステラス製薬',
 'アストラゼネカ',
 'アスペンジャパン',
 'アッヴィ',
 'アムジェン',
 'アルフレッサ ファーマ',
 'アルフレッサファーマ',
 'アールテック・ウエノ',
 'エイワイファーマ',
 'エルメッド',
 'エルメッド\u3000エーザイ',
 'エーザイ',
 'エール薬品',
 'オルガノン',
 'カイゲンファーマ',
 'ガルデルマ',
 'キッセイ薬品工業',
 'キョーリンリメディオ',
 'ギリアド・サイエンシズ',
 'クリニジェン',
 'グラクソ・スミスクライン',
 'グラクソ・スミスクライン・コンシューマー・ヘルスケア・ジャパン',
 'ケミックス',
 'コーアイセイ',
 'コーアバイオテックベイ',
 'サノフィ',
 'サンド',
 'サンドファーマ',
 'サンノーバ',
 'サンファーマ',
 'シオノギファーマ',
 'シオノケミカル',
 'スキャンポファーマ',
 'セオリアファーマ',
 'セルジーン',
 'ゼリア新薬工業',
 'ゾンネボード製薬',
 'タカミツ',
 'ダイト',
 'テイカ製薬',
 'テバ製薬',
 'トクホン',
 'トーアエイヨー',
 'ナガセ医薬品',
 'ニプロ',
 'ニプロESファーマ',
 'ニプロパッチ',
 'ニプロファーマ',
 'ニプロＥＳ',
 'ニプロＥＳファーマ',
 'ノバルティス\u3000ファーマ',
 'ノーベルファーマ',
 'バイエル薬品',
 'バクスター',
 'ビオメディクス',
 'ビーブランド・メディコーデンタル',
 'ファイザー',
 'ファイザーＵＰＪ',
 'フェリング・ファーマ',
 'フェルゼンファーマ',
 'フヂミ製薬所',
 'ブリストル・マイヤーズ',
 'ブリストル・マイヤーズ\u3000スクイブ',
 'ポーラファルマ',
 'マイラン製薬',
 'マイランＥＰＤ',
 'マルホ',


In [30]:
# test
# ここにあなたのHTMLコードを設定してください
url="https://www.iyaku.info/medicine/?sk2=ジスチグミン臭化物錠５ｍｇ「テバ」&sk3=&sf=1#st"

r=requests.get(url)
soup=BeautifulSoup(r.content,"html.parser")
# 指定されたCSSセレクターを使用して要素を取得します
target_element = soup.select_one("#medicine > div.form_search2.mt40 > table.tb_bot > tr:nth-child(1) > td.w210.pl00 > p")

# 要素が存在するか確認します
if target_element:
    # 要素のテキストを取得します
    text = target_element.get_text()
    print(text)
else:
    print("要素が見つかりませんでした")


武田テバ薬品


In [31]:
med_name=list(set(data.loc[data["メーカー名"].isna(),"医薬品名"]))
len(med_name)

276

In [32]:
name2cleaned = {}
cleaned=[]
for name in med_name:
    # クリーン処理を実行
    cleaned_name = unicodedata.normalize('NFKC', re.sub(r'\u3000', ' ', re.search(r'.*\u3000', name).group()) if re.search(r'.*\u3000', name) else name)
    
    cleaned.append(cleaned_name)
    # クリーン前のテキストをキー、クリーン後のテキストを値として辞書に追加
    name2cleaned[name] = cleaned_name
len(cleaned)

276

In [36]:
cleaned

['ロンバニンカプセル1mg',
 'ツロブテロール(2mg)貼付剤',
 'ベクロメタゾンプロピオン酸エステル8.5mg8.5g点鼻液',
 'ビダラビン軟膏3%「MEEK」',
 'チモロール点眼液T0.5%',
 'フロラーズ点鼻液50μg28噴霧用 ',
 'プルナローション2%',
 'ピアーレシロップ65%',
 'ジクロフェナクNa坐剤12.5mg「武田テバ」',
 '塩酸アンブロキソール内用液0.75%「PH」',
 'デルモゾール軟膏0.12%',
 'ポノフェン錠15mg',
 'クロピドグレル錠75mg「EE」',
 'ナフトピジルOD錠25mg「KN」',
 'ビホナゾールクリーム1%「タイヨー」',
 'ベタメタゾン酪酸エステルプロピオン酸ローション0.05%MYK',
 'ベラプロストNa錠20μg「ファイザー」',
 'テルビナフィン塩酸塩1%クリーム',
 'アシクロビル200mg錠',
 'ブテナフィン塩酸塩1%クリーム',
 'フルコナゾール100mgカプセル',
 'クロピドグレル錠75mg「ニプロ」',
 'トルナフタート液 ',
 'ラニチジン錠150mg「日医工」',
 'ビスコポール外用液1%',
 'ビホナゾール1%クリーム',
 'エプカロール錠25μg ',
 'アンブロキソール塩酸塩錠15mg「タナベ」',
 'タムスロシン塩酸塩OD錠0.2mg「ファイザー」',
 'イトラコナゾール錠50「MEEK」 ',
 '塩酸アンブロキソール錠15mg「PH」',
 'Dソルビトール原末「マルイシ」',
 'ケトコナゾール2%ローション',
 'ウテロン錠5mg',
 'ナイスピー点鼻液50μg ',
 '塩酸テルビナフィンクリーム1%「MEEK」',
 'テルビー錠125mg',
 'ビクロノール外用液1%',
 'ミリカレット点鼻液50μg28噴霧用 ',
 'シロスタゾール錠50mg「マイラン」',
 'ジクロフェナクNaゲル1%「武田テバ」',
 'ジクロフェナクナトリウム坐剤50mg「CH」',
 'ホルダゾール錠50 ',
 'ビフォノール外用液1%',
 'クロピドグレル錠75mg「ファイザー」',
 'タムスロシン塩酸塩OD錠0.1mg「ファイザー」',
 'ステリ・ネブ クロモリン吸入液1% ',
 '

In [52]:
matched=[]
for name in cleaned:
    pattern = r'「(.*?)」'
    matches = re.findall(pattern, name,flags=re.MULTILINE)
    for match in matches:
        # print(match)
        matched.append(match)
pd.Series(matched).value_counts(),len(matched)

(ファイザー    20
 KN       12
 タイヨー      9
 MEEK      8
 テバ        8
 トーワ       7
 武田テバ      6
 日医工       6
 サワイ       4
 アメル       4
 フソー       3
 TCK       3
 マルイシ      2
 ニプロ       2
 EK        2
 PH        2
 EE        2
 あすか       2
 イワキ       2
 タナベ       2
 DSEP      1
 CH        1
 10        1
 PP        1
 MYK       1
 タツミ       1
 50μg      1
 TOA       1
 日本臓器      1
 NP        1
 YD        1
 コーワ       1
 25μg      1
 マイラン      1
 イセイ       1
 JG        1
 NikP      1
 ケミファ      1
 dtype: int64,
 124)

In [34]:
# data_lack=data[data["メーカー名"].isna()]
name2firm={}
failed=[]
for i,name in enumerate(cleaned):
    url=f"https://www.iyaku.info/medicine/?sk2={name}&sk3=&sf=1#st"
    # print(url)
    r=requests.get(url)
    soup=BeautifulSoup(r.content,"html.parser")
    # 指定されたCSSセレクターを使用して要素を取得します
    target_element = soup.select_one("#medicine > div.form_search2.mt40 > table.tb_bot > tr:nth-child(1) > td.w210.pl00 > p")

    # 要素が存在するか確認します
    if target_element:
        # 要素のテキストを取得します
        text = target_element.get_text()
        name2firm[name]=text
        # print(text)
    else:
        failed.append(name)
        # print("要素が見つかりませんでした")
    print(i, end=" ")
    time.sleep(1.3)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 

In [35]:
len(name2firm)

0

In [ ]:
data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2firm)

In [18]:
data_lack=data[data["メーカー名"].isna()]
data_lack["医薬品名"].value_counts(),data_lack["薬価基準収載医薬品コード"].value_counts()

(Ｄソルビトール原末「マルイシ」        16
 Ｄソルビトール液　６５％           16
 Ｄソルビトール内用液６５％「マルイシ」    16
 Ｄソルビトール液　７５％           16
 Ｄソルビトール経口液７５％「コーワ」     16
                        ..
 ナフトピジルＯＤ錠５０ｍｇ「あすか」      1
 ナフトピジルＯＤ錠２５ｍｇ「ＫＮ」       1
 スチブロン軟膏０．０５％            1
 ビホナゾール１％液               1
 イトラコナゾール５０ｍｇ錠           1
 Name: 医薬品名, Length: 276, dtype: int64,
 2319001M1019    104
 2239001F1017     85
 2655708N1016     54
 2259004F2010     51
 1329707Q1017     45
                ... 
 2646730N1011      1
 2655702N1019      1
 2659710Q1019      1
 1319720Q3019      1
 1149117F1012      1
 Name: 薬価基準収載医薬品コード, Length: 121, dtype: int64)

In [22]:
data.shape
data.to_csv("../merged/len3_ndb_blp_DN_firm_filled.csv",encoding="utf-8")